In [1]:
##Load metadata from file

import ast
d = {}
nary = {}
with open("metadata_trial2.txt", 'r') as f:
    for line in f:
        d = (ast.literal_eval(line))
        nary.update({d['id'] : d})

In [2]:
#Filter terms that have children and aspect

filnary = {}
kilnary = {}
for key, value in nary.items():
    if value.get('children') != None and value.get('aspect') != None:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect'], 
                    'children' : value['children']})
        kilnary.update({key : filnary})

In [3]:
#import owlready2

import types
from owlready2 import *
onto = get_ontology("http://test.org/onto.owl")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [9]:
#define core classes

with onto:
    class therapeutic_agent(Thing):
        namespace = onto

In [16]:
#Making BP and MF classes recursively

def hierarchy_builder(ID, parent):
    if ID in nary and nary[ID].get('children') == None:
        with onto:
            classname = types.new_class(ID, (parent,))
    else:
        with onto:
            classname = types.new_class(ID, (parent,))
        parent = getattr(onto, ID)
        if ID in kilnary:
            for item in kilnary[ID]['children']:            
                hierarchy_builder(item['id'], parent)
    return 0

for key, value in kilnary.items():
    hierarchy_builder(key, Thing)

In [5]:
#classmaker function to make FTC classes

def classmaker(term, parentclass):
                classnameA = 'GOA' + str(term)
                classnameP = 'GOP' + str(term)
                with onto:
                    classname = types.new_class(classnameA, (parentclass,))
                    classname = types.new_class(classnameP, (parentclass,))
                return 0

In [6]:
#Using classmaker to make FTC classes

for key, value in kilnary.items():
    if value['aspect'] == 'biological_process':
        for item in value['children']:
            term = str(value['id'][3:])
            if item['relation'] == 'positively_regulates':
                classmaker(term, therapeutic_agent)
            if item['id'] in kilnary:
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('biological process', value['id'], 'positively regulates', item['id'])
            if item['relation'] == 'negatively_regulates':
                classmaker(term, therapeutic_agent)
            if item['id'] in kilnary:
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('biological process', value['id'], 'negatively regulates', item['id'])
            if item['relation'] == 'regulates':
                classmaker(term, therapeutic_agent)
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('biological process', value['id'], 'regulates', item['id'])
                
    if value['aspect'] == 'molecular_function':
        for item in value['children']:
            if item['relation'] == 'positively_regulates':
                classmaker(term, therapeutic_agent)
            if item['id'] in kilnary:
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('molecular function', value['id'], 'positively regulates', item['id'])
            if item['relation'] == 'negatively_regulates':
                classmaker(term, therapeutic_agent)
            if item['id'] in kilnary:
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('molecular function', value['id'], 'negatively regulates', item['id'])
            if item['relation'] == 'regulates':
                classmaker(term, therapeutic_agent)
            if item['id'] in kilnary:
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('molecular function', value['id'], 'regulates', item['id'])

In [7]:
#assign appropriate labels to FTC classes

x = list(onto.classes())
for y in x:
    term = 'GO:' + str(y)[8:]
    if str(y)[7] == 'A' and term in kilnary :
        y.label = 'anti_' + kilnary[term]['name'] + '_agent'
    elif str(y)[7] == 'P' and term in kilnary :
        y.label = 'pro_' + kilnary[term]['name'] + '_agent'

In [8]:
#list of ids from the created classes in the two text formats

ids = []
ontoids = []
for cl in list(onto.classes()):
    cl = str(cl)
    ontoids.append(cl)
    clt = cl[5:7] +':' + cl[8:]
    ids.append(clt)

In [9]:
#propertymaker function for making properties

def propertymaker(pname, pdomain, prange):
    with onto:
            classname = types.new_class((pname), (ObjectProperty,), )
    return 0

In [10]:
#making properties and dictionary of relations

this_thing = {}
related_to = {}

for key, value in kilnary.items():
    for item in value['children']:
        if item['id'] in ids:
            propertymaker(item['relation'], value['id'], item['id'])
            this_thing[item['relation']] = []
            related_to[item['relation']] = []
for key, value in kilnary.items():    
    for item in value['children']:
        if item['id'] in ids and value['id'] in ids:
            this_thing[item['relation']].append(ontoids[ids.index(value['id'])])
            related_to[item['relation']].append(ontoids[ids.index(item['id'])])

In [11]:
#extracting relations as Nonetype objects from respective dictionaries

for key, value in this_thing.items():
    for item in value:
        value[value.index(item)] = getattr(onto, item[5:])
        
for key, value in related_to.items():
    for item in value:
        value[value.index(item)] = getattr(onto, item[5:])

In [12]:
list_1 = []
list_2 = []

for key, value in this_thing.items():
    for item in value:
        list_1.append(item)
for key, value in related_to.items():
    for item in value:
        list_2.append(item)

In [13]:
#Making  object property alternative for is_a

with onto:
    class is_a_op(ObjectProperty):
        pass

In [14]:
#Making subproperties positively_regulates and negatively_regulates

with onto:
    class regulates(ObjectProperty):
        pass
    class positively_regulates(regulates):
        pass
    class negatively_regulates(regulates):
        pass

In [15]:
    for item in this_thing['part_of']:
        setattr(item, 'part_of', [related_to['part_of'][this_thing['part_of'].index(item)]])
    print("Finished part_ofs")
    for item in this_thing['is_a']:
        setattr(item, 'is_a', [related_to['is_a'][this_thing['is_a'].index(item)]])
    print("Finished is_as")
    for item in this_thing['regulates']:
        setattr(item, 'regulates', [related_to['regulates'][this_thing['regulates'].index(item)]])
    print("Finished regulates'")
    for item in this_thing['positively_regulates']:
        setattr(item, 'positively_regulates', [related_to['positively_regulates'][this_thing['positively_regulates'].index(item)]])
    print("Finished positively_regulates'")
    for item in this_thing['negatively_regulates']:
        setattr(item, 'negatively_regulates', [related_to['negatively_regulates'][this_thing['negatively_regulates'].index(item)]])
    print("Finished negatively_regulates'")
    for item in this_thing['occurs_in']:
        setattr(item, 'occurs_in', related_to['occurs_in'])
    print("All done.")

Finished part_ofs
Finished is_as
Finished regulates'
Finished positively_regulates'
Finished negatively_regulates'
All done.


In [18]:
onto.save(file = "testowl2", format = "rdfxml")

In [17]:
list(onto.classes())

[onto.therapeutic_agent,
 onto.GO:0000002,
 onto.GO:0033955,
 onto.GO:0032042,
 onto.GO:0000003,
 onto.GO:0032505,
 onto.GO:0019098,
 onto.GO:0032504,
 onto.GO:0061887,
 onto.GO:0022414,
 onto.GO:0019953,
 onto.GO:0003006,
 onto.GO:0045136,
 onto.GO:0048071,
 onto.GO:0032120,
 onto.GO:0080128,
 onto.GO:0010582,
 onto.GO:0009860,
 onto.GO:0080092,
 onto.GO:0001893,
 onto.GO:0046697,
 onto.GO:0007285,
 onto.GO:0048530,
 onto.GO:0010254,
 onto.GO:0060524,
 onto.GO:0010431,
 onto.GO:2000693,
 onto.GO:0010162,
 onto.GO:0048700,
 onto.GO:1902039,
 onto.GO:0048838,
 onto.GO:1902040,
 onto.GO:2000033,
 onto.GO:0010231,
 onto.GO:0098755,
 onto.GO:2000034,
 onto.GO:2000692,
 onto.GO:1990068,
 onto.GO:0048515,
 onto.GO:0010450,
 onto.GO:0010081,
 onto.GO:0048655,
 onto.GO:0060008,
 onto.GO:0090378,
 onto.GO:0009793,
 onto.GO:0010654,
 onto.GO:0048508,
 onto.GO:0010069,
 onto.GO:0009942,
 onto.GO:0048825,
 onto.GO:0010262,
 onto.GO:0048444,
 onto.GO:0048482,
 onto.GO:0035263,
 onto.GO:0034304,
 on

In [17]:
for key, value in this_thing.items():
    print(key)

part_of
is_a
regulates
positively_regulates
capable_of_part_of
negatively_regulates
capable_of
occurs_in
